## Importación de Datos

In [ ]:
ruta = '/content/drive/MyDrive/TOG/IDI III/Datos/'
df = pd.read_excel(ruta+"datos_klines.xlsx")

## Visualización y Descripción de 'datos'

In [ ]:
df.head()

In [ ]:
df.columns = ['Open_time','Open','High','Low','Close',
                'Volume','Close_time','Quote_asset_volume',
                'Number_of_trades','Taker_buy_base_asset_volume',
                'Taker_buy_quote_asset_volume','Ignore','Response']

Información de 'data'

In [ ]:
df.info()

In [ ]:
df.describe()

## Conversión 'unix'

In [ ]:
df['Open_time'] = pd.to_datetime(df['Open_time'], unit='ms', origin='unix')
df['Close_time'] = pd.to_datetime(df['Close_time'], unit='ms',origin='unix')

## Análisis Exploratorio

In [ ]:
df.hist()

### Simetría y Curtosis

In [ ]:
df.skew()

In [ ]:
df.kurt()

In [ ]:
sns.countplot(x='Response', data=df)
plt.title('Distribución de la Variable Objetivo')
plt.show()

Se puede visualizar variables altamente correlacionadas entre sí.

### Visualización de Datos

In [ ]:
sns.countplot(x='Response', data=df)
plt.show()

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df['Open_time'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'])])
fig.show()

## Dividiendo Tiempo

In [ ]:
df.set_index('Open_time', inplace = True)
df['day_of_week'] = df.index.dayofweek
df['hour_of_day'] = df.index.hour
df['month'] = df.index.month

## Datos Atípicos

In [ ]:
q1=df.quantile(0.25)
q3=df.quantile(0.75)
iqr=q3-q1
iqr

In [ ]:
sns.boxplot(data = df[['Open','High','Low','Close']])
sns.boxplot(data = df[['Volume','Number_of_trades']])
sns.boxplot(data = df[['Quote_asset_volume','Taker_buy_quote_asset_volume']])
sns.boxplot(data = df['Taker_buy_base_asset_volume'])

## Ajuste de Distribución de Probabilidad Empírica

In [ ]:
datos = df['variable']

distribuciones = ['gamma', 'expon', 'lognorm', 'beta', 'norm']
resultados_evaluacion = []
mejor_distribucion = None
mejor_parametros = None
mejor_sse = float('inf')

for distribucion in distribuciones:

    distribucion_actual = getattr(stats, distribucion)
    parametros = distribucion_actual.fit(datos)


    sse = sum((distribucion_actual.pdf(datos, *parametros) - datos)**2)


    n = len(datos)
    k = len(parametros)
    aic = n * np.log(sse / n) + 2 * k
    bic = n * np.log(sse / n) + k * np.log(n)

    resultados_evaluacion.append({
        'Distribución': distribucion,
        'SSE': sse,
        'AIC': aic,
        'BIC': bic
    })

    if sse < mejor_sse:
        mejor_distribucion = distribucion_actual
        mejor_parametros = parametros
        mejor_sse = sse

df_resultados = pd.DataFrame(resultados_evaluacion)

sns.set(style="whitegrid")


plt.figure(figsize=(10, 6))
sns.histplot(datos_taker_quote, bins=30, kde=True, color='skyblue', label='Datos', stat='density')
x = np.linspace(datos_taker_quote.min(), datos_taker_quote.max(), 100)
pdf = mejor_distribucion.pdf(x, *mejor_parametros)
plt.plot(x, pdf, 'r-', linewidth=2, label='Distribución Mejor Ajustada')
plt.xlabel('Variable')
plt.ylabel('Densidad de Probabilidad')
plt.title('Ajuste de Distribución a Variable')
plt.legend()
plt.show()